# **Tugas 2 | Web Scraping (https://pta.trunojoyo.ac.id/)**

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
wiki_link = "https://pta.trunojoyo.ac.id/c_search/byprod/10"
html = urlopen(wiki_link).read()
soup = BeautifulSoup(html, 'html.parser')

In [3]:
content_jurnal = soup.find("div", {"id": "content_journal"})


In [ ]:
# inisialisasi variable
page = 0
temp = []
index = 0
# get link pagenation
for each in content_jurnal.findAll("li"):
    bb = each.find_all("a")
    for i in bb:
      try:
        if("https://pta.trunojoyo.ac.id/c_search/byprod/" in i['href']):
          page = int(i['href'].replace('https://pta.trunojoyo.ac.id/c_search/byprod/10/',""))
      except:
        continue
# action to get data by link pagenation
for i in range(page):
  page_link = "https://pta.trunojoyo.ac.id/c_search/byprod/10/"+str(i)
  html_page = urlopen(page_link).read()
  soup_page = BeautifulSoup(html_page, 'html.parser')
  content_page = soup_page.find("div", {"id": "content_journal"})
  for page in content_page.findAll("li"):
      page_data = page.find_all("a")
      # action to get content in page
      for j in page_data:
        # print(j)
        try:
          if("https://pta.trunojoyo.ac.id/welcome/detail/" in j['href']):
            page_content = j['href']
            html_content = urlopen(page_content).read()
            soup_content = BeautifulSoup(html_content, 'html.parser')
            data_content = soup_content.find("div", {"id": "content_journal"})
            for data in data_content.findAll("li"):
              # get title
              alist = data.find_all("a")
              for a in alist:
                if('title' in a['class']):
                    temp_text = ''
                    for text in a.text.split():
                      if text != " " or text != "\r" or text != "\n":
                        temp_text+=text+" "
                    temp.append([temp_text])
              
              # get penulis
              spanlist= data.find_all("span")
              temp_penulis = ""
              for span in spanlist:
                if("Penulis : " in span.text):                  
                  temp_penulis = span.text.replace("Penulis : ","")                    
                  temp[index].append(temp_penulis)                
                
              # get abstrak
              abstrak_list = data.find_all("p")
              temp_abstrak = []
              for abstrak in abstrak_list:
                if(len(temp_abstrak) ==1):
                  continue
                else:
                  temp_abstrak.append(abstrak.text)

              temp_abstrak_text = ""          
              for text_abstrak in temp_abstrak[0].split():
                if text_abstrak != " " or text_abstrak != "\r" or text_abstrak != "\n":
                        temp_abstrak_text+=text_abstrak+" "
                else:
                        temp_abstrak_text+=text_abstrak+" "
              temp[index].append(temp_abstrak_text)
              # print(len(temp))
              index+=1
                    
        except:
          continue
print(temp)
print(len(temp))

In [ ]:
# list to dataframe
import pandas as pd
import os
path = "/content/drive/MyDrive/prosaindata/DataCrawling/Data_pta.csv"
df = pd.DataFrame(temp, columns =['Judul', 'Penulis','Abstrak'])
# add column
df['Program Studi'] = "Teknik Informatika"
# save to csv
if( os.path.exists(path) == True):
    os.remove(path)
df.to_csv(path)
df